In [1]:
import os
import sys
import json

import numpy as np
import pandas as pd
import torch

import EconDL.DataHelpers.DataProcesser as DataProcesser
import EconDL.DataHelpers.DataLoader as DataLoader
import EconDL.TrainVARNN as TrainVARNN

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

/home/isaac/.cache/pypoetry/virtualenvs/econdl-sxkRgAzp-py3.8/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/isaac/.cache/pypoetry/virtualenvs/econdl-sxkRgAzp-py3.8/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:7: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,
/home/isaac/.cache/pypoetry/virtualenvs/econdl-sxkRgAzp-py3.8/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:7: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,


In [34]:
### TOGGLE SETTINGS HERE

num_repeats = 1
num_inner_bootstraps = 5
run_name = '2aug_local_test'

run_params = {
  'var_names':  ['DGS3', 'inf', 'unrate'],
  'n_lag_linear': 1,
  'n_lag_d': 2,
  'test_size': 100,
  "precision_lambda": 0.25, 
  "lambda_temper_epochs": 40, 
  "time_dummy_setting": 2, 
  "optimizer": "Adam",
  "num_bootstrap": num_inner_bootstraps,
  "model": "VARNN"
}

experiments_params = [
  {"nodes": [400, 200, 100], "actv": "nn.ReLU()", "constant_tvpl": [50], "tvpl_archi": [5], 
    "lr": 0.001, "dropout_rate": 0.25, "s_pos": [[0, 9]], "joint_estimation": True, 
    "name": "0: ReLU Baseline"},
    {"nodes": [400, 200, 100], "actv": "nn.SELU()", "constant_tvpl": [50], "tvpl_archi": [5], 
    "lr": 0.001, "dropout_rate": 0.25, "s_pos": [[0, 9]], "joint_estimation": True, 
    "name": "0: SELU Baseline"},
]

In [35]:
with open(f'exp_config/nn_hyps_default.json', 'r') as f:
  default_nn_hyps = json.load(f)
default_nn_hyps.update(run_params)

folder_path = f'results/{run_name}'
if os.path.isdir(folder_path) == False:
  os.mkdir(folder_path)
else:
  print('Folder already exists')

Folder already exists


In [36]:
dataset, _, _ = DataLoader.load_data(dataset_name = 'monthly')

for repeat_id in range(num_repeats):
  for experiment_id, experiment_params in enumerate(experiments_params):
    # Combine the default nn_hyps with changed_nn_hyps
    nn_hyps = default_nn_hyps.copy()
    nn_hyps.update(experiment_params)

    # Process the data
    X_train, X_test, Y_train, Y_test, nn_hyps = DataProcesser.process_data_wrapper(dataset, nn_hyps)

    # Run the experiment
    results = TrainVARNN.conduct_bootstrap(X_train, X_test, Y_train, Y_test, nn_hyps, device)

    # Save the results
    results_saved = {
        'betas': results['betas_draws'], 
        'betas_in': results['betas_in_draws'], 
        'sigmas': results['sigmas_draws'], 
        'sigmas_in': results['sigmas_in_draws'],
        'precision': results['precision_draws'], 
        'precision_in': results['precision_in_draws'],
        'cholesky': results['cholesky_draws'], 
        'cholesky_in': results['cholesky_in_draws'],
        'train_preds': results['pred_in_ensemble'] , 
        'test_preds': results['pred_ensemble'], 
        'y': Y_train, 
        'y_test': Y_test, 
        'params': nn_hyps
    }

    with open(f'{folder_path}/params_{experiment_id}_repeat_{repeat_id}.npz', 'wb') as f:
      np.savez(f, results = results_saved)

/mnt/c/users/isaac/OneDrive/Documents/Isaac/2021-2022 Senior/Economic Research PGC/EconDL/EconDL/DataHelpers/DataLoader.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x_d_all['L0_HOUST'] = x_d_all['L0_HOUST'].diff()
/home/isaac/.cache/pypoetry/virtualenvs/econdl-sxkRgAzp-py3.8/lib/python3.8/site-packages/statsmodels/tsa/deterministic.py:1451: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(self._index, pd.Int64Index):
/home/isaac/.cache/pypoetry/virtualenvs/econdl-sxkRgAzp-py3.8/lib/python3.8/site-packages/statsmodels/tsa/ar_model.py:248: FutureWarning: The parameter names will change after 0.12 is released. Set old_nam

DataLoader: Loaded dataset monthly
Size of x_mat before appending MARX (737, 3)
Size of x_mat_marx (737, 6)
x_mat_all size (737, 9)
Size of X_train afer appending time (737, 70) Time dummy setting: 2
x_pos {'DGS3': [0], 'inf': [1], 'unrate': [2]}
Size of X_train (637, 70)
Bootstrap iteration 0 at time 2022-08-02 12:48:50.069100
X shape torch.Size([637, 9])
Approximate NN size (MB):  35.394775390625
OOB Mean Log Det Precision: -3.8177008628845215, MSE: 3.6038732528686523
Epoch: 0, Loss: 4.673230171203613, OOB Loss: 4.835147380828857, LR: 0.0009975000000000001, precision_lambda: 0.26
OOB Mean Log Det Precision: 2.1511600017547607, MSE: 3.019867420196533
Epoch: 40, Loss: 0.7928335666656494, OOB Loss: 2.346071481704712, LR: 0.0009024623212601519, precision_lambda: 0.01
OOB Mean Log Det Precision: 2.6512482166290283, MSE: 3.001288652420044
Epoch: 80, Loss: 0.3059701919555664, OOB Loss: 2.7055160999298096, LR: 0.0008164794398939967, precision_lambda: 0.01
Early stopped, best epoch: 43, train

/home/isaac/.cache/pypoetry/virtualenvs/econdl-sxkRgAzp-py3.8/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1217: RuntimeWarning: All-NaN slice encountered
  r, k = function_base._ureduce(a, func=_nanmedian, axis=axis, out=out,
/home/isaac/.cache/pypoetry/virtualenvs/econdl-sxkRgAzp-py3.8/lib/python3.8/site-packages/statsmodels/tsa/deterministic.py:1451: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(self._index, pd.Int64Index):
/home/isaac/.cache/pypoetry/virtualenvs/econdl-sxkRgAzp-py3.8/lib/python3.8/site-packages/statsmodels/tsa/ar_model.py:248: FutureWarning: The parameter names will change after 0.12 is released. Set old_names to False to use the new names now. Set old_names to True to use the old names. 
  warnings.warn(
/home/isaac/.cache/pypoetry/virtualenvs/econdl-sxkRgAzp-py3.8/lib/python3.8/site-packages/statsmodels/tsa/deterministic.py:1451: F

OOB Mean Log Det Precision: -0.9722981452941895, MSE: 2.356126546859741
Epoch: 0, Loss: 4.839632987976074, OOB Loss: 3.5301387310028076, LR: 0.0009975000000000001, precision_lambda: 0.26
OOB Mean Log Det Precision: 2.793877601623535, MSE: 1.9614359140396118
Epoch: 40, Loss: 1.2417945861816406, OOB Loss: 0.5481760501861572, LR: 0.0009024623212601519, precision_lambda: 0.01
OOB Mean Log Det Precision: 2.887808322906494, MSE: 1.948314905166626
Epoch: 80, Loss: 1.0232765674591064, OOB Loss: 0.6068792343139648, LR: 0.0008164794398939967, precision_lambda: 0.01
OOB Mean Log Det Precision: 2.8073489665985107, MSE: 2.0456621646881104
Epoch: 120, Loss: 0.8992354869842529, OOB Loss: 0.47473907470703125, LR: 0.0007386886522184711, precision_lambda: 0.01
OOB Mean Log Det Precision: 2.9116404056549072, MSE: 2.038663625717163
Epoch: 160, Loss: 0.7622432708740234, OOB Loss: 0.7060067653656006, LR: 0.0006683094493930968, precision_lambda: 0.01
Early stopped, best epoch: 120, train loss: 0.899235486984

/home/isaac/.cache/pypoetry/virtualenvs/econdl-sxkRgAzp-py3.8/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1217: RuntimeWarning: All-NaN slice encountered
  r, k = function_base._ureduce(a, func=_nanmedian, axis=axis, out=out,


In [5]:
dataset.shape

(739, 7)